DONE

# Volumetric meshes of zebrafish brain geometry

In [ ]:
import sys
sys.path.append("..")

from main import *
from visualization import *

import numpy as np
import lapy
from lapy import TetMesh, Solver, TriaMesh
import matplotlib.pyplot as plt
import numpy as np
import skimage.measure
import pygalmesh
import meshio
from scipy.ndimage import zoom
import os

# 1. Generating optic tectum meshes

In [ ]:
from scipy.sparse import load_npz
from scipy.ndimage import gaussian_filter, zoom

In [ ]:
masks = load_npz('/home/anleg84/Documents/Atlas/Mapzebrain/' + 'region_masks.npz') # Binary region masks of the mapZebrain atlas

In [ ]:
region_id = 22 # Region id of the tectum periventricular layer

mask_full = np.flip(np.reshape(masks[:, region_id].toarray(), [359, 974, 597], order='C'), axis=0)
mask_left = np.copy(mask_full)
mask_left[:, :, 284:] = 0 
mask_right = np.copy(mask_full)
mask_right[:, :, :284] = 0

#### Meshing left hemisphere

In [ ]:
vol = zoom(mask_left, (0.25, 0.25, 0.25), order=1).astype('bool').astype('uint16') # Downscaling for speed, then upscaling

In [ ]:
voxel_size = (0.1, 0.1, 0.1)

mesh = pygalmesh.generate_from_array(
    vol, voxel_size, max_facet_distance=0.1, max_cell_circumradius=0.1
)

mesh.write("../Files/tectum_left.vtk")

#### Meshing right hemisphere

In [ ]:
vol = zoom(mask_right, (0.25, 0.25, 0.25), order=1).astype('bool').astype('uint16')

In [ ]:
voxel_size = (0.1, 0.1, 0.1)

mesh = pygalmesh.generate_from_array(
    vol, voxel_size, max_facet_distance=0.1, max_cell_circumradius=0.1
)

mesh.write("../Files/tectum_right.vtk")

#### Meshing both hemispheres

In [ ]:
smoothed = gaussian_filter(mask_full, (10, 10, 10))

In [ ]:
np.sum(smoothed > 111)

In [ ]:
np.sum(mask_full > 0)

In [ ]:
vol = zoom(smoothed > 111, (0.25, 0.25, 0.25), order=0).astype('bool').astype('uint16')

In [ ]:
voxel_size = (0.1, 0.1, 0.1)

mesh = pygalmesh.generate_from_array(
    vol, voxel_size, max_facet_distance=0.1, max_cell_circumradius=0.1
)

mesh.write("../Files/tectum_full.vtk")

# 2. Computing tectal eigenmodes

#### Left tectum

In [ ]:
mesh = meshio.read('../Files/tectum_left.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=100)

Plotting eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenvectors[:, i+1], alpha=0.5, cmap='coolwarm')
    #ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1])
    #ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
plt.show()

In [ ]:
np.save('../Files/tectum_vertices_left.npy', vertices)
np.save('../Files/tectum_eigenmodes_left.npy', eigenvectors.T)

#### Right tectum

In [ ]:
mesh = meshio.read('../Files/tectum_right.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=100)

Plotting eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 2], vertices[:, 1], -vertices[:, 0], c=eigenvectors[:, i + 1], alpha=0.5, cmap='coolwarm')
    #ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1])
    #ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
plt.show()

In [ ]:
np.save('../Files/tectum_vertices_right.npy', vertices)
np.save('../Files/tectum_eigenmodes_right.npy', eigenvectors.T)

#### Full tectum

In [ ]:
mesh = meshio.read('../Files/tectum_full.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=100)

Plotting eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 2], vertices[:, 1], -vertices[:, 0], c=eigenvectors[:, i + 1], alpha=0.5, cmap='coolwarm')
    #ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1])
    #ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
plt.show()

In [ ]:
np.save('../Files/tectum_vertices_full.npy', vertices)
np.save('../Files/tectum_eigenmodes_full.npy', eigenvectors.T)

# Computing tectum nodes by fragmenting masks

In [ ]:
from scipy.sparse import load_npz
from scipy.ndimage import gaussian_filter, zoom
from sklearn.cluster import KMeans

In [ ]:
masks = load_npz('/home/anleg84/Documents/Atlas/Mapzebrain/' + 'region_masks.npz')

In [ ]:
region_id = 22

mask_full = np.flip(np.reshape(masks[:, region_id].toarray(), [359, 974, 597], order='C'), axis=0)

mask_left = np.copy(mask_full)
mask_left[:, :, 284:] = 0

mask_right = np.copy(mask_full)
mask_right[:, :, :284] = 0

#### Left hemisphere

In [ ]:
mask_downsampled = zoom(mask_left, (0.5, 0.5, 0.5), order=1).astype('bool').astype('uint16')

In [ ]:
i, j, k = np.where(mask_downsampled)
voxels = np.stack([k, j, i], axis=1).astype('float')

In [ ]:
n_clusters = 400

In [ ]:
kmeans = KMeans(n_clusters=n_clusters).fit(voxels)
clusters = kmeans.labels_

centroids = []
for c in np.unique(clusters):
    centroids.append(np.mean(voxels[clusters == c], axis=0))
centroids = np.stack(centroids, axis=0)

In [ ]:
nodes_tectum = centroids

In [ ]:
np.save('../Files/nodes_tectum_left.npy', nodes_tectum)

#### Right hemisphere

In [ ]:
mask_downsampled = zoom(mask_right, (0.5, 0.5, 0.5), order=1).astype('bool').astype('uint16')

In [ ]:
i, j, k = np.where(mask_downsampled)
voxels = np.stack([k, j, i], axis=1).astype('float')

Here, the commented `n_clusters` reflect the different coarse-graining resolutions used in Supplementary Figure S8**a**.

In [ ]:
#n_clusters = 200
# n_clusters = 400
n_clusters = 800
# n_clusters = 800
# n_clusters = 1000 

In [ ]:
kmeans = KMeans(n_clusters=n_clusters).fit(voxels)
clusters = kmeans.labels_

centroids = []
for c in np.unique(clusters):
    centroids.append(np.mean(voxels[clusters == c], axis=0))
centroids = np.stack(centroids, axis=0)

In [ ]:
nodes_tectum = centroids

In [ ]:
np.save('../Files/nodes_tectum_right_800.npy', nodes_tectum)

# Generating whole-brain mesh

#### Both hemispheres

In [ ]:
mask = np.load('../Files/mask_atlas.npy')

In [ ]:
volume = zoom(mask, (0.2, 0.2, 0.2), order=1)

voxel_size = (1, 1, 1)

mesh = pygalmesh.generate_from_array(
    volume, voxel_size, max_facet_distance=2.5, max_cell_circumradius=1.5
)

mesh.write("../Files/mesh_zebrafish_wholebrain.vtk")

#### Single hemisphere

In [ ]:
mask[:, :, 284:] = 0
volume = zoom(mask, (0.2, 0.2, 0.2), order=1)

voxel_size = (1, 1, 1)

mesh = pygalmesh.generate_from_array(
    volume, voxel_size, max_facet_distance=2.5, max_cell_circumradius=1.5
)

mesh.write("../Files/mesh_zebrafish_wholebrain_single.vtk")

# Computing whole-brain eigenmodes

#### Both hemispheres

In [ ]:
mesh = meshio.read("../Files/mesh_zebrafish_wholebrain.vtk")

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenmodes = solver.eigs(k=100)

Plotting eigenmodes:

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 2], vertices[:, 1], -vertices[:, 0], c=eigenmodes[:, i + 1], alpha=0.5, cmap='coolwarm')
    #ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1])
    #ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
plt.show()

In [ ]:
np.save('../Files/zebrafish_wholebrain_eigenmodes.npy', eigenmodes.T)
np.save('../Files/zebrafish_wholebrain_vertices.npy', np.flip(vertices, axis=1) * 5)

#### Single hemisphere

In [ ]:
mesh = meshio.read("../Files/mesh_zebrafish_wholebrain_single.vtk")

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenmodes = solver.eigs(k=100)

Plotting eigenmodes:

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 2], vertices[:, 1], -vertices[:, 0], c=eigenmodes[:, i + 1], alpha=0.5, cmap='coolwarm')
    #ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1])
    #ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
plt.show()

In [ ]:
np.save('../Files/zebrafish_wholebrain_eigenmodes_single.npy', eigenmodes.T)
np.save('../Files/zebrafish_wholebrain_vertices_single.npy', np.flip(vertices, axis=1) * 5)